In [12]:
import sys
sys.path.insert(1, '/home/ec2-user/SageMaker/github/aspect_topic_modeling')
import torch
from torch.nn.functional import normalize
import pickle 
import torch.nn.functional as F 
import numpy as np 
from sklearn.metrics import f1_score
from torch import nn, optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pickle
import random
from scipy import sparse
import itertools
from scipy.io import savemat, loadmat
import re
import string
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import pandas as pd
import swifter
from src.models.utils import remove_stopWords
from src.models.utils import get_wordnet_pos, remove_stopWords, get_emb, generate_emb, train, kld_normal, get_common_words, generate_bow
from collections import Counter
from models.NVDM import VNTM, topic_covariance_penalty, sinkhorn_torch, NTM, negative_sampling_prior, optimal_transport_prior,  NormalParameter, get_mlp, EmbTopic, NSSTM, OTETM
from src.models.utils import get_wordnet_pos, remove_stopWords, get_emb, generate_emb, train, kld_normal, get_common_words
from hyperspherical_vae.distributions import VonMisesFisher
from hyperspherical_vae.distributions import HypersphericalUniform
from src.features.metric import diversity, get_topic_coherence,top_purity
from sklearn.cluster import KMeans
from collections import Counter
from sklearn import metrics
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.matutils import Sparse2Corpus

In [45]:
def evaluate(topics, X, z, labels):
    result = []
    result += sorted(diversity(topics))


    labels_pred = torch.argmax(z, 1).numpy()
    labels_true = labels
    #coherence_score = get_topic_coherence(X.toarray(), topics, word_track)
    kmeans = KMeans(n_clusters=numb_embeddings, random_state=0).fit(z.detach().numpy())
    result += [top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)]  
    corpus = Sparse2Corpus(X, documents_columns=False)
    #decoder_weight = self.autoencoder.decoder.linear.weight.detach().cpu()
    id2word = {index: str(index) for index in range(X.shape[1])}
    texts = df.apply(lambda x:[str(i) for i in x['index_num']], axis = 1).values
    cm = CoherenceModel(
                topics=topics,
                corpus=corpus,
                dictionary=Dictionary.from_corpus(corpus, id2word),
                coherence="u_mass",
            )
    result += [cm.get_coherence()]
    cm = CoherenceModel(
        topics=topics,
        texts = texts,
        corpus=corpus,
        dictionary=Dictionary.from_corpus(corpus, id2word),
        coherence='c_npmi',
    )

    result += sorted(cm.get_coherence_per_topic())

    cm = CoherenceModel(
        topics=topics,
        texts = texts,
        corpus=corpus,
        dictionary=Dictionary.from_corpus(corpus, id2word),
        coherence='c_v',
    )

    result += sorted(cm.get_coherence_per_topic())
    return result

def train(model, X, batch_size, epoch, optimizer, scheduler):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    total_nll = 0.0
    total_kld = 0.0
    total_words = 0
    total_penalty = 0.0
    #size = epoch_size * batch_size
    indices = torch.randperm(X.shape[0])
    indices = torch.split(indices, batch_size)
    #print(indices)
    length = len(indices)
    for idx, ind in enumerate(indices):
        data_batch = torch.from_numpy(X[ind].toarray()).float().to(device)
        
        d = model(x = data_batch)
            
        
        
        total_nll += d['rec_loss'].sum().item() / batch_size
        total_kld += d['kld'].sum().item() / batch_size  
        #total_penalty += d['prior']  
#         if i < 3:
#             loss = d['minus_elbo']
#         else:
        loss = d['loss']

        optimizer.zero_grad()
        loss.sum().backward()
        optimizer.step()
        scheduler.step()
    print(total_nll/length, total_kld/length)
     


In [62]:
data = fetch_20newsgroups(subset = 'all')
df = pd.DataFrame()
df['class'] = data['target']
df['text'] = data['data']
df['text'] = df.swifter.apply(lambda x: ' '.join(remove_stopWords(x['text'])), axis=1)
df['clean_text']  = df.swifter.apply(lambda x: x['text'].split(), axis = 1)

common_words_ct = Counter([j for i in df['clean_text'].values for j in i])
common_words = get_common_words(common_words_ct, ct = 200)
word_track = {i: ind for ind, i in enumerate(common_words)}
index_track = {ind: i for ind, i in enumerate(common_words)}
df['index_num'] = df.apply(
            lambda x: [word_track[i] for i in x['clean_text'] if i in word_track], axis=1)
#change it to any location you save your embeddings
vec = '/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/20news/jose.txt'
embed = generate_emb(vec, common_words).cpu()
X, indices = generate_bow(df = df, common_words = common_words)
labels = df['class'].values

Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

In [44]:
pdf = []
for penalty in [0.1, 0.5, 1, 2]:
            result = [penalty] 
            
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            numb_embeddings = 20
            layer = 64
            hidden = get_mlp([X.shape[1], layer], nn.ReLU)
            normal = NormalParameter(layer, numb_embeddings)
            h_to_z = nn.Softmax()
            embedding = nn.Embedding(X.shape[1], 50)
            # p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
            # out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

            embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
            embedding.weight.requires_grad=False
            topics = EmbTopic(embedding = embedding,
                              k = numb_embeddings, normalize = False)



            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


            model = VNTM(hidden = hidden,
                        normal = normal,
                        h_to_z = h_to_z,
                        topics = topics,
                        layer = layer, 
                        top_number = numb_embeddings,
                        penalty = penalty
                        ).to(device).float()
            # larger hidden size make topics more diverse
            #num_docs_train = 996318
            batch_size = 256
            optimizer = optim.Adam(model.parameters(), 
                                   lr=0.002, 
                                   weight_decay=1.2e-6)

            epochs = 20
            scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.05, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

            
            for epoch in range(epochs):
                train(model, X,  batch_size, epoch, optimizer, scheduler)
            #Add Diversity
            emb = model.topics.get_topics().cpu().detach().numpy()
            topics =  [[str(ind) for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(10)]
                #Add purity . 
            data_batch = torch.from_numpy(X.toarray()).float()
            model.cpu()
            z = model.hidden(data_batch)
            z_mean = model.fc_mean(z)
            z_mean = z_mean / z_mean.norm(dim=-1, keepdim=True)
            z = model.h_to_z(z_mean)

            result += evaluate(topics, X, z, labels)
            pdf += [result]
            pd.DataFrame(pdf).to_csv('vntm_result.csv')
            

/home/ec2-user/SageMaker/github/aspect_topic_modeling/models/NVDM.py:518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = self.h_to_z(10 * z)


852.3764339137722 0.0003528987258874081
833.9358751451647 0.0065030172810185
802.7080605996622 0.020629807450884097
776.9980361526077 0.02769756314621584
766.5454596442145 0.02930298065011566
762.1701288996516 0.029433899433226197
759.6222435203758 0.028382767017024593
757.1568570523649 0.02808703759030716
755.48470986856 0.027972861016924318
753.9910995895798 0.027753485945632327
753.1505753800676 0.026251285286569916
752.4136715450802 0.025921327993273735
752.4556769293707 0.026337043130518618
751.680248363598 0.02590395215697385
751.3044062434016 0.02674811937519022
751.2801703375738 0.025424336788018007
751.1537071434227 0.025275769920365232
750.8851887470967 0.025157848171688414
750.8321689915012 0.025106761478693097
750.8933418892525 0.024943803411883278


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:55: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


TypeError: list indices must be integers or slices, not str

In [66]:
class GSM(NTM):
    def __init__(self, hidden, normal, h_to_z, topics, penalty):
        # h_to_z will output probabilities over topics
        super(GSM, self).__init__(hidden, normal, h_to_z, topics)
        self.penalty = penalty

    def forward(self, x, n_sample=1):
        stat = super(GSM, self).forward(x, n_sample)
        loss = stat['loss']
        penalty, var, mean = topic_covariance_penalty(self.topics.topic_emb)

        stat.update({
            'loss': loss + penalty * self.penalty,
            'penalty_mean': mean,
            'penalty_var': var,
            'penalty': penalty * self.penalty,
        })

        return stat

In [71]:
from models.NVDM import Topics
import gc
pdf = []
#labels = [[120], [1527], [1646], [2047], [727], [1624], [36], [32], [26], [92], [907], [652]]
for numb_embeddings in [10, 20, 30, 40, 50]:
    result = [numb_embeddings] 
    torch.cuda.empty_cache()
    gc.collect()
    hidden = get_mlp([X.shape[1], 64], nn.ReLU)
    normal = NormalParameter(64, numb_embeddings)
    h_to_z = nn.Softmax()
    embedding = nn.Embedding(X.shape[1], 50)
    # p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
    # out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

    embedding.weight = torch.nn.Parameter(torch.ones(embed.shape))
    # embedding.weight.requires_grad=False
    #embedding.weight = torch.nn.Parameter()
    embedding.weight.requires_grad=True
    topics = EmbTopic(embedding = embedding,

                      k = numb_embeddings, normalize = False)




    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    model = GSM(hidden = hidden,
                normal = normal,
                h_to_z = h_to_z,
                topics = topics,
                penalty = 0.5
                ).to(device).float()
    # larger hidden size make topics more diverse
    #num_docs_train = 996318
    batch_size = 256
    optimizer = optim.Adam(model.parameters(), 
                           lr=0.002, 
                           weight_decay=1.2e-6)



    epochs = 20
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.002, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

    for epoch in range(epochs):
        train(model, X,  batch_size, epoch, optimizer, scheduler)
    emb = model.topics.get_topics().cpu().detach().numpy()
    topics =  [[str(ind) for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(numb_embeddings)]
    data_batch = torch.from_numpy(X.toarray()).float()
    model.cpu()
    h = model.hidden(data_batch)
    h = model.drop(h)
    mu, log_sigma = model.normal(h)
    z = model.h_to_z(mu)
    result += evaluate(topics, X, z, labels)
    pdf += [result]
    pd.DataFrame(pdf).to_csv('gsm_result.csv')

840.2764043034734 0.14752137429408124
815.4568110285578 2.7103528992549792
783.0952944368929 4.3609197590802165
778.784373515361 2.8426911959777006
774.6264524717589 2.0234160681028626
767.9100135597023 10077.176793913584
761.0873215134079 69.3053106391752
758.6336665797878 17459687.533643696
755.7382354736328 5.221328803010889
754.188020448427 7.864280893995955
752.6771248482369 5.630689608084189
752.2076655207453 5.81131914821831
751.0493918753959 1054.950396782643
750.3951333535684 292.8652725284164
749.6096240894215 5.54470435348717
749.1894704457876 5.585414751155956
749.0334604624155 5.618317694277377
749.0159070813978 5.692249317426939
748.7631118362015 5.63982825021486
748.6938946698162 5.757757908589131


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[862, 666, 666, 418, 796, 908, 528, 37, 1]
[637, 280, 443, 235, 245, 644, 624, 506, 312, 349]


/home/ec2-user/SageMaker/github/aspect_topic_modeling/models/NVDM.py:76: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = self.h_to_z(z)


841.3765431996939 0.2074762776091292
824.7175342456715 1.6799932181633808
788.926717397329 4.4504952172975285
778.4003398998364 2.7038868504601554
771.5081828349346 22.919909606108796
767.6175396893475 81561227.25958209
765.020475645323 381745.9069249888
762.3192369615709 22.672403786633467
760.3181300807644 1901.11142216502
758.405460254566 91.92166087434099
757.46695028769 89.50418971035931
755.9209384402714 1051.522579908371
755.8784357019373 6.126181818343498
754.3259034027924 6.314897157050468
753.2341923069309 6.495797630902883
752.9509260847761 6.121557989635983
752.813206028294 6.317051104597144
752.3209310995566 6.2774295678009855
752.0234399743982 9.539597479072777
752.6269539497994 429.3934269499134


...

/home/ec2-user/SageMaker/github/aspect_topic_modeling/models/NVDM.py:76: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = self.h_to_z(z)


842.1229982118349 0.29041738848428467
828.2131252804318 1.469753977817458
791.1007075954128 4.826619865121068
778.4166028821791 2.2966643225502326
772.4511759989971 56.343130382331644
766.5601905616554 3.090942531018644
762.3672790527344 12.06990261657818
760.5549910261824 122.45083993995512
757.7920952874262 73.35032663796399
756.0032690924567 9.046773143716761
755.075959901552 4422.890766237233
754.3145310685442 39526.52317291659
753.1285577722498 12.265821118612546
752.3087467502903 6.469494764869277
751.9834025614971 6.108860354165773
751.1051111994562 6.484751063424188
750.6703293259079 6.450681032361211
750.5690233385241 6.543433969085281
749.3702285354202 6.530648663237288
750.9291431323902 199604022.4133276


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[889, 650, 960, 573, 138, 575, 621, 544, 8, 15, 1, 1]
[188, 127, 135, 246, 323, 448, 289, 159, 146, 255, 274, 270, 60, 172, 92, 200, 116, 79, 160, 144, 142, 106, 24, 73, 54, 81, 208, 71, 41, 13]


KeyboardInterrupt: 

In [64]:
result

[0.1]

[852, 114, 444, 222, 13, 10, 45, 3, 3, 1, 5, 6, 1, 1, 1, 1, 1]
[922, 9, 165, 247, 39, 40, 25, 6, 3, 2, 8, 4, 3, 1, 2, 5, 3, 1, 4, 1]


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.09142523612437653,
 0.05306401815449709,
 0.07906186989281545,
 0.04809175504402425,
 -2.1699192849931492,
 0.08258238579931092,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931093,
 0.08258238579931095,
 0.08258238579931095,
 0.08258238579931095,
 0.08258238579931095,
 0.08258238579931095,
 0.536016927230133,
 0.536016927230133,
 0.536016927230133,
 0.536016927230133,
 0.536016927230133,
 0.536016927230133,
 0.536016927230133,
 0.536016927230133,
 0.536016927230133,
 0.536016927230133,
 0.536016927230133,
 0.536016927230133,
 0.5360169272301331,
 0.5360169272301331,
 0.5360169272301331,
 0.5360169272301331,
 0.536

In [42]:
evaluate(topics, X)

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[893, 837, 845, 582, 289, 821, 371, 712, 719, 1, 15]
[800, 156, 702, 711, 153, 408, 176, 245, 811, 429, 611, 598, 161, 89, 132, 125, 180, 144, 206, 87]


[0.24,
 0.24,
 0.52,
 0.52,
 0.52,
 0.56,
 0.6,
 0.76,
 0.92,
 1.0,
 0.3228801867770349,
 0.41982885388588964,
 0.367398917542184,
 0.418928198549303,
 -2.5155681338841136,
 -0.004246315719902591,
 0.009250011116884164,
 0.015947860063648848,
 0.016623999768910807,
 0.024740270344067478,
 0.031372713131173256,
 0.03817326942023648,
 0.16384449351879934,
 0.19023430200125338,
 0.23307599784639488,
 0.3544211477680313,
 0.4221171970530066,
 0.42948688114679656,
 0.43567113607794133,
 0.49566456333515096,
 0.5224178117319039,
 0.576256403596454,
 0.6970673372046872,
 0.7332098326412175,
 0.7452441090914286]

In [36]:
np.mean(result[:10])

0.588

In [35]:
 np.mean([-0.004246315719902591,
 0.016623999768910807,
 0.015947860063648848,
 0.03817326942023648,
 0.031372713131173256,
 0.23307599784639488,
 0.024740270344067478,
 0.009250011116884164,
 0.19023430200125338,
 0.16384449351879934])

0.0719016601491466